In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import numpy as np
import itertools
import dpa_load
import multiprocessing as mp
from datetime import datetime
from itertools import repeat
import split_articles
import numeric_articles
import continue_articles
# we import some functions from the Handelsblatt folder
import sys
sys.path.insert(1, os.getcwd().replace('dpa code', 'Handelsblatt'))
import count_words_mp
import identify_eng_2
import correct_url
import clean_dpa_articles
import clean_dpa_references

In [2]:
# Set the number of cores to use
NUM_CORE = mp.cpu_count()-4

# DPA Data (1991 - 2018)

Deutsche PresseAgentur (DPA) is the Germany's biggest news agency which sells its news reports to the leading German newspapers. We believe that the data set has a high chance to be useful for economic forecasting because DPA produces information that is timely and has a large reach.

We purchased DPA data in November 2019. The corpus consists of **7,539,874** articles from January 1991 to December 2018.

The data set includes news from both dpa-Basisdienstes and dpa-afx Wirtschaftsnachrichten. The former one is the basic news service covering such topics as Economy, Politics, and  Finance. The second one was created in 1999. It specializes in financial news.

## Load the data

First, we read in the data by extracting the following XML elements:

* title - article's title
* text - text of the article
* date - publication date
* ressort - section (Politics vs Economy)
* source/credit - source (dpa vs afx)
* city - which city the news article refers to
* genre - journalistic genre, e.g., chronology, story, table
* wortanzahl - word count
* keywords - keywords associated with an article

In [3]:
# Folder with unpacked articles
#path = r'E:\\Userhome\\jbaer\\dpa_unpacked'

#path = r'G:\\Test\\Results\\dpa Raw Data\\dpa_unpacked'
path = os.getcwd().replace('\\newspaper_data_processing\\dpa code', '') + '\\dpa_unpacked'

folder_list = []

# 2 folders for dpa and dpa-afx 
for fol in [fol for fol in os.listdir(path)]:

    # Within each folder: folders for different years
    for f in [f for f in os.listdir(path + '\\' + fol)]:
        folder_list.append(path + '\\' + fol + '\\' + f)

In [4]:
# Select a path to the folder for storing results
#PATH = r'G:\\Test\\Results'
#os.chdir(PATH)

In [5]:
# Use the 'dpa_load' function to load articles
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    df_list = pool.map(dpa_load.dpa_load, folder_list)
    data = pd.concat(df_list)
    data.reset_index(inplace=True, drop=True)
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:25:04.926592


In [6]:
print(len(data))

7539874


In [7]:
data.head()

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic
0,Hessischer Energieversorger EAM senkt Strompre...,16147043.xml,17,8,1999,wi,dpa-afx,PRD,Hessischer Energieversorger EAM senkt Strompre...,KASSEL,,127,afx
1,Europäische Bonds: Bei dünnem Handel leichter....,16147044.xml,17,8,1999,wi,dpa-afx,BND CLO,Europäische Bonds: Bei dünnem Handel leichter.,LONDON,,110,afx
2,US-Paketzusteller UPS stellt für Steuerstreit ...,16147045.xml,17,8,1999,wi,dpa-afx,FNG,US-Paketzusteller UPS stellt für Steuerstreit ...,ATLANTA,,131,afx
3,"Burlington kauft kanadische Erdgasfirma für 2,...",16147046.xml,17,8,1999,wi,dpa-afx,MNA,"Burlington kauft kanadische Erdgasfirma für 2,...",HOUSTON,,68,afx
4,BNP bewertet Consors nach Halbjahresergebnis w...,16147047.xml,17,8,1999,wi,dpa-afx,RTG,BNP bewertet Consors nach Halbjahresergebnis w...,FRANKFURT,,97,afx


In [8]:
#data.to_csv('dpa_raw.csv')

In [3]:
data = pd.read_csv('dpa_raw.csv', encoding = 'utf-8', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'})

In [4]:
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [5]:
data.head()

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,5739189.xml,1,1,1991,pl,dpa,Schalck-Golodkowski,Schalck: Milliardenkredit sicherte Zahlungsfäh...,Berlin,,218,WiPo
1,Tschads Regierung: Bevölkerung soll Waffen abl...,5739191.xml,1,1,1991,pl,dpa,Tschad,Tschads Regierung: Bevölkerung soll Waffen abl...,N'Djamena,,75,WiPo
2,Welajati: Iran bleibt bei einem Krieg am Golf ...,5739193.xml,1,1,1991,pl,dpa,Golfkrise Iran,Welajati: Iran bleibt bei einem Krieg am Golf ...,Teheran,,90,WiPo
3,Bush will offenbar seinen Außenminister erneut...,5739195.xml,1,1,1991,pl,dpa,Golfkrise USA,Bush will offenbar seinen Außenminister erneut...,Washington,,181,WiPo
4,Morgenzusammenfassung Neue Runde diplomatische...,5739199.xml,1,1,1991,pl,dpa,Golfkrise,Morgenzusammenfassung Neue Runde diplomatische...,Washington/Luxemburg,,504,WiPo


# Pre-processing

## Light pre-processing

### Remove short articles (<100 words)

Short articles are often incoherent or contain only insiginicant news. For this reason we decided to filter out articles that consist of less than 100 words. 

In [6]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:01:34.731927


In [7]:
# Save the result as a new column "word_count"
data['word_count'] = count_results

In [8]:
# remove articles with less than 100 words
data = data[data['word_count']>=100]
data.reset_index(inplace=True, drop=True)
print(len(data))

5366317


In [9]:
#data.to_csv('dpa_prepro_step1.csv')

### Remove exact duplicates

Some articles are more than once in the corpus. We filter out all duplicates and only keep the articles with the oldest date.

In [10]:
data.drop_duplicates(['texts'], keep = 'first', inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4542012


In [11]:
#data.to_csv('dpa_prepro_step2.csv')

## Filtering

Unfortunately, dpa articles are not as consistently sorted into sections and subsections as articles from other 
news media. Instead, we investigate the most commonly used titles and keywords and remove non-economic articles based on them.

We exclude non-economic and other irrelevant articles based on the following titles

* 1) Londoner Edelmetallpreise: Precious metal prices (without text)
* 2) Tageskalender: List of upcoming events
* 3) (Tabelle): Tables in text form
* 4) SPORT, Sport (except for the titles that contain TRANSPORT, INTERSPORT, PASSPORT, Sportartikel, news about sportswear manufacturers, sporting goods industry, SPORTARTIKLER, Sportmodelle, or Sportswear): news related to Sports. Beware that news about sports marketing agencies (e.g., Sportfive), sports media websites (e.g., Sportal), and new sports models of car manufacturers might be removed as well. 
* 5) Berichtigung, KORREKTUR (except for the titles that contain Berichtigungsaktien, rectified shares): Article corrections
* 6) Impressum: Dpa contact data
* 7) Testmeldung: Test-articles from dpa
* 8) Kurse A, Kurse B, Kurse C, Kurse D, KURSE A, KURSE B, KURSE C, KURSE DREI, KURSE Drei, Kurse drei: Stock charts without text
* 9) DGAP-DD: DGAP reports
* 10) New Yorker Aktien-Schlußkurse: Stock closing prices at the New York stock exchange (articles only occur from 1997 to 2002)
* 11) VERMISCHTES: Miscellaneous with no relation to economics
* 12) Angekündigte US-Quartalszahlen auf einen Blick: Quarterly US figures

In [12]:
# Filter out non-economic articles based on titles.
fil_titles = '''Londoner Edelmetallpreise|Tageskalender|\(Tabelle\)|SPORT|Sport|Berichtigung|KORREKTUR|Impressum|Testmeldung|Kurse A[^a-z]|Kurse B[^a-z]|Kurse C[^a-z]|Kurse D[^a-z]|KURSE A|KURSE B|KURSE C|KURSE DREI|Kurse drei|KURSE Drei|Kurse/drei|DGAP-DD|New Yorker Aktien-Schlusskurse|VERMISCHTES|Angekündigte US-Quartalszahlen'''
titles_exc = '''TRANSPORT|INTERSPORT|PASSPORT|Berichtigungsaktien|Sportartikel|SPORTARTIKLER|Sportmodelle|Sportswear'''
data.drop(data[(data['title'].str.contains(fil_titles, na = False)) & (~data['title'].str.contains(titles_exc, na=False))].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4207086


We exclude non-economic articles based on the following sections.

* 1) Tabelle: Tables in text form (some articles are still left after the previos step)
* 2) Historisches: News about historical events
* 3) Achtung: Announcemt of upcoming news

In [13]:
# Filter out non-economic articles based on sections.
fil_genres = '''Tabelle|Historisches|Achtung'''
data.drop(data[data['genre'].str.contains(fil_genres, na = False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4183903


We exclude non-economic articles based on the following keywords.
* 1) Redaktionshinweis: Editor's notes for Dpa journalists
* 2) DGAP: DGAP reports
* 3) Sport, SPORT, SPO (except for Sportartikel, this section contains articles on sports companies): Sport news (some sports articles are still left after the previos steps)
* 4) Kurse A, Kurse B, Kurse C, Kurse D, KURSE A, KURSE B, KURSE C, Kurse D,
     KURSE DREI, Kurse drei, KURSE Drei, KURSE drei, Kurse Drei, Kurse/drei: Stock charts without text (some articles are  still left after the previos steps)
* 5) Tagesvorschau, Vorschau, VORSCHAU, vorschau: List of titles of upcoming news
* 6) Bilderdienst: Dpa Picture Service
* 7) Geschichte: News related to historical events

In [14]:
# Filter out non-economic articles based on keywords.
fil_keywords = '''Redaktionshinweis|DGAP|Sport|SPORT|SPO|Kurse A|Kurse B|Kurse C|Kurse D|KURSE A|KURSE B|KURSE C|KURSE DREI|Kurse drei|KURSE Drei|KURSE drei|Kurse Drei|Kurse/drei|Kurse D|Tagesvorschau|Vorschau|vorschau|VORSCHAU|Bilderdienst|Geschichte'''
keywords_exc = '''Sportartikel'''
data.drop(data[(data['keywords'].str.contains(fil_keywords, na = False)) & (~data['keywords'].str.contains(keywords_exc, na=False))].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3926157


We exclude articles based on the following bits of text.
* 1) Schalterverkaufskurse: Precios metal prices
* 2) dpa-news.de: News regarding the Dpa website
* 3) Wirtschafts- und Finanztermine, Wirtschafts- und Finanz-Termine, Konjunktur- und Wirtschaftstermine: List of dates when economic data will be published/economic events will take place
* 4) DGAP (except for articles that contain DGAP standing for Deutsche Gesellschaft für Auswärtige Politik): DGAP reports
* 5) Bitte verwenden Sie diese Meldung nicht: Retracted articles
* 6) \( Wiederholung: Repeated articles
* 7) [§] 26 Abs., § 15a WpHG 1, § 15 WpHG, Artikel 19 MAR, article 19 Market Abuse Regulation (MAR): Regulatory news
* 8) Die Pivotpunkte für den Dax-Future: Pivot points for the Dax-Future
* 9) An der Frankfurter Wertpapierbörse wurden, Die Aktien im Dow Jones EuroStoxx 50, Die Aktien im Dow Jones Euro Stoxx 50: Stock charts
* 10) Ihr Ansprechpartner: Redaktion Politik International: List of current political news headlines
* 11) (Achtung - Sonderdisposition): List of articles on a particular topic

In [9]:
fill_text = '''Schalterverkaufskurse:|dpa-news\.de|Wirtschafts- und Finanztermine|Wirtschafts- und Finanz-Termine|DGAP|Bitte verwenden Sie diese Meldung nicht|Konjunktur- und Wirtschaftstermine|\(Wiederholung|[§] 26 Abs\. 1|§ 15a WpHG|§ 15 WpHG|Artikel 19 MAR|article 19 Market Abuse Regulation \(MAR\)|Die Pivotpunkte für den Dax-Future|An der Frankfurter Wertpapierbörse wurden|Die Aktien im Dow Jones EuroStoxx 50|Die Aktien im Dow Jones Euro Stoxx 50|\(Achtung - Sonderdisposition\)'''
text_exc = '''Auswärtige Politik'''
data.drop(data[(data['texts'].str.contains(fill_text, na = False)) & (~data['texts'].str.contains(text_exc, na=False))].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

C:\Users\Nutzer\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


3511650


We want to exclude non-economic articles based on the following two sources.
* 1) dpa-frei: Article corrections
* 2) dpa-wahl: Articles about federal election results

In [10]:
data.drop(data[data['source'].str.contains('dpa-frei|dpa-wahl', na = False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3511626


We exclude articles regarding dpa itself.

In [11]:
data.drop(data[data['city'].str.contains('Die Deutsche Presse-Agentur', na = False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3511622


In [13]:
#data.to_csv('dpa_prepro_step3.csv')

### Split up articles

Sometimes multiple articles are collected and merged into one entry. For example, articles with the title, keyword, or genre 
'Nachrichtenüberblick' are a collection of the most important articles of the day. Because these smaller articles
can have different sentiments and topics, we separate articles that consist of multiple smaller articles. Articles consisting of multiple smaller articles can be identified with the following words which can appear in titles, keywords, or genres.

- dpa-Nachrichtenüberblick, Nachrichtenüberblick: List of news for the upcomming days or from news which are a few days old
- Vorschau, vorschau, VORSCHAU: List of news for the upcomming days
- Die Woche in Berlin (1999-2018), Die Woche in Bonn (1991-1999): News regarding the parlament from last week. 
- Aus der Landespolitik - Kurz gemeldet: Collection of short political articles regarding regional politics
- Die politische Woche: Announcments of Political (and Economical) news for next week
- Kurznachrichten Wirtschaft: Collection of short economic news
- Wochenvorschau: List of news for the upcoming week
- Analysten-Einstufungen, ANALYSTEN-EINSTUFUNGEN:
- Notizen aus der Politik, NOTIZEN AUS DER POLITIK, Notizen aus der Politik: Collection of varios short political articles 


In [23]:
s_mult_art = '''dpa-Nachrichtenüberblick|Nachrichtenüberblick|Die Woche in Berlin|Die Woche in Bonn|Aus der Landespolitik - kurz gemeldet|Die politische Woche|Kurznachrichten Wirtschaft|Wochenvorschau|Analysten-Einstufungen|ANALYSTEN-EINSTUFUNGEN|Notizen aus der Politik|NOTIZEN AUS DER POLITIK'''
mult_art = data[data['title'].str.contains(s_mult_art, na = False)]
mult_art = mult_art.append(data[data['keywords'].str.contains(s_mult_art, na = False)])
mult_art = mult_art.append(data[data['genre'].str.contains(s_mult_art, na = False)])
mult_art.drop_duplicates(['texts'], keep = 'first', inplace=True)
mult_art.reset_index(inplace=True, drop=True)

In [24]:
# delete 'mult_art' from the original data
data.drop(data[data['title'].str.contains(s_mult_art, na = False)].index, inplace=True)
data.drop(data[data['keywords'].str.contains(s_mult_art, na = False)].index, inplace=True)
data.drop(data[data['genre'].str.contains(s_mult_art, na = False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)

In [25]:
# calculate chunck size 
chunk_size = int(mult_art.shape[0]/NUM_CORE)

# split data into chunks 
chunks = [mult_art.iloc[mult_art.index[i:i + chunk_size]] for 
          i in range(0, mult_art.shape[0], chunk_size)]

In [26]:
# split up articles in into smaller articles and append the resulting new articles 
# to the corpus
from datetime import datetime
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    results = pool.map(split_articles.split_articles, chunks) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

results = pd.concat(results)
print(len(results))

0:07:44.690301


The separated articles consist of fewer words than the articles from which they originally stemmed. Therefore, we count the number of words of the new articles with the count_words_mp function from before and filter out articles with less than 100 words.

In [27]:
# count the number of words for the seperated articles and filter out articles with less
# than 100 words
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in results['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:09.189036


In [28]:
results['word_count'] = count_results
results = results[results['word_count']>=100]

In [29]:
# append seperated articles to corpus
data = data.append(results)
print(len(data))

2031724


In [30]:
#data.to_csv('dpa_prepro_step4.csv')

### Identify and Delete English Articles

In [37]:
# Delete all English articles from the data  
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    eng_results = pool.map(identify_eng_2.identify_eng_2, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

1:48:39.715753


In [48]:
data['language'] = eng_results
data = data[data.language==0]
data.reset_index(inplace=True, drop=True)
print(len(data))

2064333


### Identify articles that predominantly consist of numbers

Articles that consist predominately of numbers sometimes carry little sentiment. Filtering out all numbers only helps a little, because the resulting texts are often grammatical nonsensical. To get a better understanding of how these articles look we identify all articles that consist of more than 80% of numbers.

In [31]:
# use the 'numeric_articles' function to identify economic articles with a high share of numbers in them
inputs = zip(data['texts'], data['word_count'], itertools.repeat(0.80))

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    numeric_list = pool.starmap(numeric_articles.numeric_articles, inputs)
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:01:23.476436


In [32]:
numeric = data[numeric_list]

In [33]:
# inspect example article with high share of numbers
numeric['texts'].iloc[0]

'FLENSBURG (dpa-AFX) - Der langjährige Renner VW-Golf <VOW.ETR> ist auch 1999 an der Spitze der Auto-Neuzulassungen in Deutschland geblieben. Den 356.000 Golf-Exemplaren - einschließlich seiner Abwandlungen Vento und Bora - folgten erneut rund 230.000 Opel-Astra <GM.NYS>. Das geht aus einer Statistik der Kraftfahrt-Bundesamts (KBA) in Flensburg hervor, die am Dienstag veröffentlicht wurde. Insgesamt kamen 1999 knapp über 3,8 Millionen fabrikneue Personenwagen in den Straßenverkehr. 1998 waren es 3,73 Millionen.      Auf den dritten Platz liegen mit 143.000 Erstanmeldungen die Fahrzeugtypen der Dreier-Reihe von BMW <BME.ETR>, die den ebenfalls seit Jahren mit führenden VW-Passat (136.500) auf den vierten Rang verdrängten. Neuling in der 1999er Tabellenspitze der 25 Wagentypen - von insgesamt über 70 - sind der Ford-Focus <F.NYS> auf Platz fünf mit 113.000 Neuzulassungen, der VW-Lupo (16./61.000), der Peugeot 206 (22./44.000) sowie der Opel-Zafira (25./38.500). Auf Erfolgskurs ist auch d

In [34]:
numeric['texts'].iloc[1]

'Operative Kosten                        40.616     48.002    18,2% Vertriebs- und allg.                    25.892     36.359    40,4% Verwaltungskosten Zentrale Verwaltungskosten              16.114     18.884    17,2% Nicht liquiditätswirksame                    0      5.393 Personalkosten aus der Gewährung von Aktienoptionen Abschreibungen und Amortisierung       122.614    119.848    -2,3% ------------------------------------------------------------------ --- Betriebsaufwand                        205.236    228.486    11,3% ------------------------------------------------------------------ --- ------------------------------------------------------------------ --- Betriebsergebnis (-verlust)            -21.193    -21.268     0,4%  ------------------------------------------------------------------ --- Zinsaufwand                            -55.901    -31.046   -44,5% Anderer Aufwand                         -1.298     -1.500    15,6% --------------------------------------------------

In [35]:
len(numeric)

5412

### Merge continuations of articles

Some dpa-afx articles are split into multiple entries marked by the word 'Fortsetzung' at the beginning of the texts of following entries.

In [38]:
data['texts'].iloc[1]

'FRANKFURT (dpa-AFX) - Der deutsche Aktienmarkt hat am Montag für wenige Augenblicke seinen Rekordschluss vom Juli 1998 überschritten. Mit einem Höchststand von 6.188,68 lag der Dax über dem Tageschluss vom 21. Juli 1998, als der Index 6.186 Punkte erreichte. Der Dax <DAX.ETR> schloss den Handel am ersten Wochentag bei 6.142,19 Zählern und damit um 0,38% oder 23,02 Punkte fester ab. Der Nebenwerteindex M-Dax <MDAX.ETR> gab dagegen auf 3.990,00 Punkte oder um 1,43% nach, und der Neue Markt-Index Nemax 50 <NMKX.ETR> schloss bei 4.610,66 Zählern (-0,58%).      Spätestens auf dem Niveau des All-Time-High bei ungefähr 6.200 Punkten werde der Index auf einen massiven Widerstand stoßen, heißt es am Montag in einem Research-Report von der Nürnberger Schmidt Bank. Michael Schubert, Analyst der Bankgesellschaft Berlin, sagte in einem Gespräch mit dpa-AFX, derzeit sei der einzig negative Faktor, dass die Börse nicht von der Breite getragen werde, sondern nur von wenigen Titeln. Wer jetzt noch nic

In [39]:
data['texts'].iloc[3]

'(Fortsetzung) - Der Tagesgewinner SAP <SAP.ETR> übernahm bereits am Vormittag die Führung bei den Gewinnern ein und schloss den Handel mit einem Kurszuwachs von 6,27% auf 475,20 Euro ab. Händler waren sich einig darüber, dass SAP weiteres Wachstumpotenzial haben. Ende vergangener Woche habe es Kaufempfehlungen für das Papier gegeben, sagte ein Händler. Die ABN Amro Bank hatte ihr Kursziel mit 600 Euro festgesetzt.      Den Wert umgaben zudem Gerüchte über eine bevorstehende Allianz mit einem Weltkonzern, hieß es. Ein anderer Händler hielt den aktuellen SAP-Kurs noch immer für zu billig. Wachstumswerte stünden weiter in der Gunst der Anleger. Bei SAP hätten Image und Aktienkurs unter Problemen in den USA gelitten. Das werde nach Erwartung des Händlers aber spätestens mit den Geschäftszahlen des ersten Quartals 2000 überwunden sein. Mit dem Ende der Angst vor dem Jahr-2000-Problem würde der Kurs weiter steigen.     Ihren Kurshöhenflug fortsetzen konnten zudem die Aktien von Siemens <SIE

In [40]:
data['texts'].iloc[7]

'(Fortsetzung) - Überraschend fest tendierten am Montag die Autowerte. Volkswagen <VOW.ETR> gewannen 1,6% auf 49,35 Euro. Vielleicht sehen wir jetzt tatsächlich eine Branchen-Rotation zur Autobranche hin, sagte ein Händler der DG-Bank. Die nahe Zukunft des Wertes stehe oder falle mit der 50 Euro Marke. Auch BMW <BMW.ETR> (28,10 Euro/+1,04%) und DaimlerChrysler <DCX.ETR> (67,25 Euro/+0,67%)  verbuchten Kurszuwächse. Alle drei deutschen Hersteller verbuchten einem Pressebericht zufolge im Gesamtjahr 1999 steigende Absatzzahlen in den USA.       Die Verliererliste führten am Montag Lufthansa <LHA.ETR> (22,30 Euro/-3,88%) und Veba <VEB.ETR> (45,47 Euro/-3,87%) an. Unter Gewinnmitnahmen litt laut Händlern der Kurs der T-Aktie <DTE.ETR>: Der Titel verlor im Montagshandel 1,31% auf 59,30 Euro. Wäre die Telekom mit dem Indextrend im Wert gestiegen, hätte der Dax den Rekordstand vom Juli 1998 eingestellt, rechnete ein Händler am Nachmittag vor./mr/fs'

We identify which entries belong together and merge them to one article

In [46]:
# Divide data into roughly equal sized chunks where articles from one day only fall under the same chunk
data_cont = data[data['topic'] == 'afx']
dates = data_cont.groupby(['year', 'month', 'day'])
dates = list(dates.groups)
dates = np.array_split(dates, NUM_CORE)

meta_data = data_cont.loc[:, data_cont.columns != 'texts']

chunks = [pd.concat([data_cont[(data_cont['year'] == t[0]) & (data_cont['month'] == t[1]) & (data_cont['day'] == t[2])] for t in tup]) for tup in dates]
chunks = [[chunk, meta_data] for chunk in chunks]

In [48]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    continue_results = pool.map(continue_articles.continue_articles, chunks) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:14:56.636717


In [49]:
data.drop(list(itertools.chain(*[tup[0] for tup in continue_results])), inplace=True)

In [ ]:
continue_articles = pd.concat([tup[1] for tup in continue_results])
print(len(continue_articles))

In [50]:
data = data.append(continue_articles)
data.reset_index(inplace=True, drop=True)
print(len(data))

1974426


In [51]:
#data.to_csv('dpa_prepro_step6.csv')

## Remove URLs

In [52]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    url_corrected = pool.map(correct_url.correct_url, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:06:33.551105


In [53]:
data['texts'] = url_corrected

## Remove dpa references

We remove dpa references (e.g. NEW YORK (dpa) - ...) from each article.

In [54]:
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    dpa_ref_removed = pool.map(clean_dpa_references.clean_dpa_references, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:15:15.393896


In [55]:
data['texts'] = dpa_ref_removed

In [66]:
#data.to_csv('dpa_prepro_step7.csv')

# Correct spelling

### Umlauts

Older articles (1991 - 2000) from the section 'Kommentar' (Commentary) are often missing umlauts and correct capitalization. To
fix these two issues, we use the notebook Umlauts_fix written in Python 2 and the notebook Truecasing written in Python 3.

In [6]:
umlauts = ['ä', 'ö', 'ü', 'ß', 'Ä', 'Ö', 'Ü']
umlauts_replace = ['ae', 'oe', 'ue', 'ss', 'AE', 'OE', 'UE']

In [7]:
dpa_umlauts_fix = data[(data.texts.str.contains('|'.join(umlauts_replace))) & (~data.texts.str.contains('|'.join(umlauts))) & (data.year<2001)]

In [9]:
dpa_umlauts_fix['texts'].iloc[0]

'Ad-hoc announcement sent by DGAP. The sender is solely responsible for the contents of this announcement.  Ad-hoc Mitteilung Nach @ 15 WpHG  WizCom Technologies Ltd. (WizCom) <WZM.FSE> (Neuer Markt:WZM,WKN:915 856) veroeffentlicht das Ergebnis fuer das am 31. Dezember 1999 endende Geschaeftsjahr 20. Maerz 2000, Jerusalem, Israel - Der Umsatz belief sich im Jahr 1999 auf US$ Mio. 11,613. Der Vorjahreswert lag bei US$ Mio. 15,799. Der Umsatz im ersten bzw. zweiten Halbjahr 1999 betrug US$ Mio. 4,046 bzw. US$ Mio. 7,567. Das Unternehmen sieht den Umsatzrueckgang als voruebergehend an und vor allem im zweiten Halbjahr begruendet, weil das neue Produkt, der QuickLink-Pen, erst mit einigen Monaten Verzoegerung auf den Markt gebracht werden konnte. Im 4. Quartal 1999 konnte WizCom den QuickLink Pen in den USA erfolgreich auf den Markt bringen. Im 1. Quartal 2000 vermarktet WizCom den QuickLink Pen in weiteren Laendern, unter anderem Australien, Grossbritannien, Deutschland und Frankreich. De

In [ ]:
dpa_umlauts_fix.to_csv('dpa_umlauts_fix.csv', encoding='utf-8-sig', sep = ';')

In [ ]:
dpa_umlauts_fixed = pd.read_csv('dpa_umlauts_fixed.csv', encoding = 'utf-8', sep=';')

### Truecasing

In [56]:
dpa_cases_fix = data[data.texts.str.contains('^(?!.*[A-Z])')]

In [57]:
dpa_cases_fix['texts'].iloc[0]

'new york (vwd) - enttaeuschend verlief das geschaeft am mittwoch, dem ersten handelstag im neuen jahr, an der new yorker aktienboerse. die zunaechst gesehenen leichten gewinne konnten nur bis in das fruehe nachmittagsgeschaeft behauptet werden. in den letzten 2-1/2 geschaeftststunden gerieten die kurse in die minuszone und wall street schloss auf breiter front schwaecher. der dow-jones-index fuer 30 industriewerte gab um 23,02 auf 2.610,64 punkte nach. auch die uebrigen marktbestimmenden indizes gerieten in die minuszone. bei einem umsatz von 126,28 (114,13) millionen aktien standen die kursverlierer den -gewinnern im verhaeltnis von rund neun zu sieben gegenueber. verantwortlich fuer die schwaeche waren wiederauflebende befuerchtungen ueber eine anhaltende rezessionsphase. nachdem sogar das weisse haus jetzt von einer rezessionaeren entwicklung spricht, hielten sich die meisten anleger mit ihren engagements zurueck, wodurch der vorherrschende abgabedruck ausreichte, um die kurse in d

In [62]:
dpa_cases_fix.to_csv('dpa_case_fix.csv', encoding='utf-8-sig', sep = ';')

In [63]:
dpa_cases_fixed = pd.read_csv('dpa_cases_fixed.csv', encoding = 'utf-8', sep=';')

In [64]:
data.loc[dpa_cases_fixed.index, 'texts'] = dpa_cases_fixed

In [65]:
# fixed version
data['texts'].iloc[0]

'New York( Vwd)- Enttaeuschend verlief das Geschaeft am Mittwoch, dem ersten Handelstag im neuen Jahr, an der New Yorker Aktienboerse. Die Zunaechst Gesehenen leichten Gewinne konnten nur bis in das Fruehe Nachmittagsgeschaeft behauptet werden. In den letzten 2-1/2 Geschaeftststunden gerieten die Kurse in die Minuszone und Wall Street schloss auf breiter Front Schwaecher. Der Dow-Jones-Index Fuer 30 Industriewerte gab um 23,02 auf 2.610,64 Punkte nach. Auch die Uebrigen Marktbestimmenden Indizes gerieten in die Minuszone. Bei einem Umsatz von 126,28( 114,13) Millionen Aktien standen die Kursverlierer den -Gewinnern im Verhaeltnis von rund neun zu sieben Gegenueber. Verantwortlich Fuer die Schwaeche waren Wiederauflebende Befuerchtungen Ueber eine anhaltende Rezessionsphase. Nachdem sogar das Weisse Haus jetzt von einer Rezessionaeren Entwicklung spricht, hielten sich die meisten Anleger mit ihren Engagements Zurueck, wodurch der vorherrschende Abgabedruck ausreichte, um die Kurse in di

## Fixing tokens containing a number and a word

In quite a few cases, a number and a word are erroneously merged into a single token. Splitting these tokens into two tokens helps us to deal with the following problems:

(see Handelsblatt notebook)

In [58]:
startTime = datetime.now()

import split_number_word

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    split_corrected = pool.map(split_number_word.split_number_word, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:02:13.158789


In [59]:
data['texts'] = split_corrected

In [60]:
#data.to_csv('dpa_prepro_step8.csv')

# Delete Fuzzy Duplicates

In [61]:
types_keep = ['russische Aktienmarkt', 'europäischen Börsen', 'Deutsche Börse' ,'Europäische Zentralbank',
'Moskauer Aktienmarkt', 'deutsche Aktienmarkt', 'deutschen Aktienmarkt', 'Folgende Investmentbanken',
'deutsche Rentenmarkt', 'amerikanischen Treasury Bonds', 'IRW-PRESS', 'Deutsche Bank', 'Ausgewählte Analysten-Einstufungen',
'Deutsche Staatsanleihen', 'Der japanische Aktienmarkt']

exceptions = ['NO_EXCEPTIONS']

In [63]:
# import a function that outputs the indices of duplicates 
import fuzzy_duplicates
delete_indices = []
startTime = datetime.now() 
for year in list(set(data['year'])):
    data_input = data[(data['year'] == year)]
    for month in list(set(data_input[data_input['year'] == year]['month'])): # old: list(set(data))
            # Prepare inputs
            inputs_year = []
            inputs_month = []
            inputs_month_year = []
            inputs_year.append(year)
            inputs_month.append(month)
            inputs_month_year.append(data_input[(data_input['year'] == year) & (data_input['month'] == month)][["month", "year", "texts"]])

            #from itertools import repeat
            inputs = list(zip(inputs_year, inputs_month, inputs_month_year))
            from datetime import datetime
            if __name__ == "__main__":
                pool = mp.Pool(NUM_CORE)
                # apply function to all combinations of month-year in parallel
                delete_intermediate = pool.starmap(fuzzy_duplicates.fuzzy_duplicates, zip(inputs, repeat(types_keep), repeat(exceptions)))
                delete_indices = delete_indices + delete_intermediate # create one list of indices
                pool.close()
                pool.join()  
    print(year)
print(datetime.now()-startTime)

1991.0
1992.0
1993.0
1994.0
1995.0
1996.0
1997.0
1998.0
1999.0
2000.0
2001.0
2002.0
2003.0
2004.0
2005.0
2006.0
2007.0
2008.0
2009.0
2010.0
2011.0
2012.0
2013.0
2014.0
2015.0
2016.0
2017.0
2018.0
6:58:11.847691


In [64]:
delete_indices = list(set([item for sublist in delete_indices for item in sublist]))

In [65]:
data.drop(delete_indices, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

1580324


In [66]:
#data.to_csv('dpa_prepro_step9.csv')

# Clean articles

Dpa articles include some unnecessary text passages like inquiry notes or references to webpages. We decided to clean the 
affected articles from these text passages to make the sentiment classification easier for our model(s).

We remove the following terms and sections from the texts:
* 1) stock symbols
* 2) additional metadata in the text meant for the author
* 3) references to previos articles
* 4) references to dpa and dpa-AFX webpage
* 5) uncorrected original article on which a correction is based on
* 6) inquiry notes
* 7) reference to english article on which some articles are based on
* 8) date of the article
* 9) references for aditional information (phone numbers, webpages etc.)
* 10) references to sender
* 11) references to Debitos
* 12) references to issuer
* 13) reference to authors
* 14) references to summary of article 

In [67]:
startTime = datetime.now()
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    cleaned_articles = pool.map(clean_dpa_articles.clean_dpa_articles, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:58:52.745164


In [68]:
data['texts'] = cleaned_articles

In [69]:
#data.to_csv('dpa_prepro_final.csv')